# 3D Object Detection Evaluation

In [ ]:
from av2.evaluation.detection.eval import evaluate
from av2.evaluation.detection.utils import DetectionCfg
from pathlib import Path
from av2.utils.io import read_feather, read_all_annotations

In [ ]:
dataset_dir = Path.home() / "data" / "datasets" / "av2" / "sensor"  # Path to AV2 sensor dataset.
competition_cfg = DetectionCfg()

In [ ]:
split = "val"
gts = read_all_annotations(dataset_dir=dataset_dir, split=split)  # Contains all annotations in a particular split.

dts_path = Path("detections.feather")
dts = read_feather(dts_path)
dts, gts, metrics = evaluate(dts, gts, cfg=competition_cfg)  # Evaluate instances.

In [ ]:
display(metrics)